# メソッド

メソッドは引数の型が厳密に一致する順番に優先的に呼び出される。順番がまったく同一であるときは `MethodError` が投げられる。

In [9]:
f(x::Float64, y::Float64) = 2x + y
f(x::Number, y::Number) = 2x - y
f

f (generic function with 2 methods)

In [10]:
methods(f)

# 2 methods for generic function "f":
f(x::Float64, y::Float64) in Main at In[9]:1
f(x::Number, y::Number) in Main at In[9]:2

In [11]:
f(1.0, 2.0)

4.0

In [12]:
f(1, 2)

0

In [13]:
f(x,y) = println("Whoa there, Nelly.")

f (generic function with 3 methods)

In [14]:
f(1, 2)

0

In [15]:
f("1", 2)

Whoa there, Nelly.


In [16]:
g(x::Float64, y) = 2x + y
g(x, y::Float64) = x + 2y
g(2.0, 3.0)

LoadError: [91mMethodError: g(::Float64, ::Float64) is ambiguous. Candidates:
  g(x, y::Float64) in Main at In[16]:2
  g(x::Float64, y) in Main at In[16]:1
Possible fix, define
  g(::Float64, ::Float64)[39m

In [17]:
g(x::Float64, y::Float64) = 2x + 2y

g (generic function with 3 methods)

In [18]:
g(2.0, 3)

7.0

In [19]:
g(2, 3.0)

8.0

In [20]:
g(2.0, 3.0)

10.0

## パラメトリックメソッド

型パラメータを使うと巧妙なことができる。

In [21]:
same_type(x::T, y::T) where {T} = true
same_type(x,y) = false
methods(same_type)

# 2 methods for generic function "same_type":
same_type(x::T, y::T) where T in Main at In[21]:1
same_type(x, y) in Main at In[21]:2

In [22]:
same_type(1, 2)

true

In [23]:
same_type(1.0, 2)

false

## World age

メソッドの追加や再定義は「それが定義された時点」では参照不可能である。現実の因果関係に近い。

In [24]:
function tryeval()
    @eval newfun() = 1
    newfun()
end

tryeval (generic function with 1 method)

In [25]:
tryeval()

LoadError: [91mMethodError: no method matching newfun()
The applicable method may be too new: running in world age 21817, while current world is 21818.[0m
Closest candidates are:
  newfun() at In[24]:2 (method too new to be called from this world context.)[39m

In [26]:
newfun()

1

上の例では、 `newfun` そのものは生成されており、 `tryeval` からは即座に参照可能になっているのだが、それ以外のあらゆる者からは参照不可能である（ただしREPLからは呼び出せる）。 `tryeval` 実行開始時には、 world age 的にはまだ `newfun` が存在していないとみなされるため。

## パラメータ制限のある Varargs メソッド

この `bar` 関数は varargs (可変個数引数) x が2個のAny型引数でなければならない。

In [1]:
bar(a,b,x::Vararg{Any,2}) = (a,b,x)

bar (generic function with 1 method)

In [2]:
bar(1,2,3)

LoadError: [91mMethodError: no method matching bar(::Int64, ::Int64, ::Int64)[0m
Closest candidates are:
  bar(::Any, ::Any, ::Any, [91m::Any[39m) at In[1]:1[39m

In [3]:
bar(1,2,3,4)

(1, 2, (3, 4))

In [4]:
bar(1,2,3,4,5)

LoadError: [91mMethodError: no method matching bar(::Int64, ::Int64, ::Int64, ::Int64, ::Int64)[0m
Closest candidates are:
  bar(::Any, ::Any, ::Any, ::Any) at In[1]:1[39m

また、次の関数は AbstractArray A の次元と varargs indexes の個数が一致するときのみ呼び出される。

In [8]:
getindexfunc(A::AbstractArray{T,N}, indexes::Vararg{Number,N}) where {T,N} = true

getindexfunc (generic function with 1 method)

本当の関数は `getindex`。

In [36]:
getindex([["A", "B"],["C", "D"]], 1)

2-element Array{String,1}:
 "A"
 "B"

In [37]:
getindex([["A", "B"],["C", "D"]], 2)

2-element Array{String,1}:
 "C"
 "D"

In [38]:
[["A", "B"],["C", "D"]][1]

2-element Array{String,1}:
 "A"
 "B"

In [39]:
[["A", "B"],["C", "D"]][2]

2-element Array{String,1}:
 "C"
 "D"

In [40]:
[["A", "B"],["C", "D"]][1][2]

"B"

`getindex` が再帰的に呼び出されているっぽい。